## Import packages

In [1]:
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
es = Elasticsearch([{'host':'uct2-es-door.mwt2.org', 'port':9200}])

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime
%matplotlib inline
plt.style.use('ggplot')
pd.options.mode.chained_assignment = None  # default='warn'

## Choose date range and pull ES data

In [2]:
ind="stashcp-2016.8"
#undirected
myquery1={
  "query": {
    "filtered": {
      "query": {
        "query_string": {
          "query": "*",
          "analyze_wildcard": True
        }
      },
      "filter": {
        "bool": {
          "must": [
            {
              "query": {
                "query_string": {
                  "analyze_wildcard": True,
                  "query": "*"
                }
              }
            },
            {
              "range": {
                "timestamp": {
                  "gte": 1472130846947,
                  "lte": 1472150123020,
                  "format": "epoch_millis"
                }
              }
            }
          ],
        }
      }
    }
  },
}

#undirected
myquery2={
  "query": {
    "filtered": {
      "query": {
        "query_string": {
          "query": "*",
          "analyze_wildcard": True
        }
      },
      "filter": {
        "bool": {
          "must": [
            {
              "query": {
                "query_string": {
                  "analyze_wildcard": True,
                  "query": "*"
                }
              }
            },
            {
              "range": {
                "timestamp": {
                  "gte": 1468002935971,
                  "lte": 1468005422807,
                  "format": "epoch_millis"
                }
              }
            }
          ],
        }
      }
    }
  },
}

In [4]:
page= es.search(index=ind, body=myquery1, scroll='2m', search_type='scan', size=1)

sid = page['_scroll_id']

scroll_size = page['hits']['total']
results=[]

while (scroll_size > 0):
    page = es.scroll(scroll_id = sid, scroll = '2m')
    results.append(page['hits']['hits'])
    sid = page['_scroll_id']
    scroll_size = len(page['hits']['hits'])

scroll_size = page['hits']['total']
Res=[]
for i in range(0,scroll_size):
    Res.append(results[i][0]['_source'])
    
print('Number of records: ', scroll_size)

df = pd.DataFrame(Res)
#print(df.head(2))

del df['destination_space']
del df['xrdcp_version']
#del df['tries']
del df['download_size']
del df['filesize']
#del df['IP']
#del df['xrdcp_exit']

begin=[]
for i in range(scroll_size):
    begin.append((int(df['timestamp'][i])-int(df['download_time'][i]))) #convert to minutes
    
df['begin']=begin
print(list(df.columns.values))
print(df.head(1))
df.to_pickle('august25.pkl')


Number of records:  21763
['IP', 'cache', 'download_time', 'end1', 'end2', 'filename', 'host', 'sitename', 'start1', 'start2', 'start3', 'status', 'timestamp', 'tries', 'xrdexit1', 'xrdexit2', 'xrdexit3', 'begin']
                IP                        cache  download_time           end1  \
0  128.105.245.173  root://stash.osgconnect.net          10949  1472135046335   

   end2                                           filename  \
0     0  /user/shenker/public/deploy/deploy.b12ab0a.000...   

                          host  sitename         start1  start2  start3  \
0  root://stash.osgconnect.net  GLOW-OSG  1472135035386       0       0   

    status      timestamp  tries  xrdexit1  xrdexit2  xrdexit3          begin  
0  Success  1472135046000      1         0        -1        -1  1472135035051  


In [5]:
begin=int(df['begin'].iloc[1])//1000
print(df['begin'].iloc[1])
print(begin)
print (datetime.datetime.fromtimestamp(begin).strftime('%Y-%m-%d %H:%M:%S'))
#print (datetime.datetime.fromtimestamp(df['begin'].iloc[1]).strftime('%Y-%m-%d %H:%M:%S'))

1472133224989
1472133224
2016-08-25 08:53:44


## Formate datetime and save dataframe into pickle

In [6]:
length=df.shape[0]

for i in range(length):
    begin=int(df['begin'].iloc[i])//1000
    end=int(df['timestamp'].iloc[i])//1000
    start1=int(df['start1'].iloc[i])//1000
    end1=int(df['end1'].iloc[i])//1000
    start2=int(df['start2'].iloc[i])//1000
    end2=int(df['end2'].iloc[i])//1000
    start3=int(df['start3'].iloc[i])//1000
    df['begin'].iloc[i]=datetime.datetime.fromtimestamp(begin).strftime('%Y-%m-%d %H:%M:%S')
    df['timestamp'].iloc[i]=datetime.datetime.fromtimestamp(end).strftime('%Y-%m-%d %H:%M:%S')  
    df['start1'].iloc[i]=datetime.datetime.fromtimestamp(start1).strftime('%Y-%m-%d %H:%M:%S')
    df['start2'].iloc[i]=datetime.datetime.fromtimestamp(start2).strftime('%Y-%m-%d %H:%M:%S')
    df['start3'].iloc[i]=datetime.datetime.fromtimestamp(start3).strftime('%Y-%m-%d %H:%M:%S')
    df['end1'].iloc[i]=datetime.datetime.fromtimestamp(end1).strftime('%Y-%m-%d %H:%M:%S')
    df['end2'].iloc[i]=datetime.datetime.fromtimestamp(end2).strftime('%Y-%m-%d %H:%M:%S')


    
df=df.set_index(['timestamp'])    
df.index = pd.to_datetime(df.index, unit='s')
df['begin']=pd.to_datetime(df['begin'], unit='s')
df['start1']=pd.to_datetime(df['start1'], unit='s')
df['start2']=pd.to_datetime(df['start2'], unit='s')
df['start3']=pd.to_datetime(df['start3'], unit='s')
df['end1']=pd.to_datetime(df.end1, unit='s')
df['end2']=pd.to_datetime(df.end2, unit='s')
df
#df.to_pickle('august25.pkl')
print ('done')


done


## Loop to find concurrent jobs

In [ ]:
jobs = pd.concat([pd.Series(1, df.begin), pd.Series(-1, df.index)]).resample('1Min', how='sum').cumsum()
index=jobs.index

df2=pd.DataFrame(columns=['index', 'Origin', 'Nebraska', 'UCSD', 'US-MWT2_UIUC', 'BNL ATLAS', 'FZU', 'Trunk'])
finish=pd.DataFrame(columns=['index', 'Origin', 'Nebraska', 'UCSD', 'US-MWT2_UIUC', 'BNL ATLAS', 'FZU', 'Trunk'])

df2['index']=index
finish['index']=index

length2=df2.shape[0]
df2['Origin']=0
df2['Nebraska']=0
df2['UCSD']=0
df2['US-MWT2_UIUC']=0
df2['BNL ATLAS']=0
df2['FZU']=0
df2['Trunk']=0
finish['Origin']=0
finish['Nebraska']=0
finish['UCSD']=0
finish['US-MWT2_UIUC']=0
finish['BNL ATLAS']=0
finish['FZU']=0
finish['Trunk']=0
finish['Trunk Timeout']=0
finish['BNL Timeout']=0


for i in range(length2):
    for j in range(length):
        if df['tries'].iloc[j]=="1":
            if df['begin'].iloc[j]<=df2['index'].iloc[i] and df.index[j]>=df2['index'].iloc[i]:
                if df['host'].iloc[j]=='root://data.ci-connect.net':
                    df2['Origin'].iloc[i]+=1
                if df['host'].iloc[j]=='root://hcc-stash.unl.edu':
                    df2['Nebraska'].iloc[i]+=1
                if df['host'].iloc[j]=='root://mwt2-stashcache.campuscluster.illinois.edu':
                    df2['US-MWT2_UIUC'].iloc[i]+=1
                if df['host'].iloc[j]=='root://xrd-cache-1.t2.ucsd.edu':
                    df2['UCSD'].iloc[i]+=1
                if df['host'].iloc[j]=='root://osgxroot.usatlas.bnl.gov':
                    df2['BNL ATLAS'].iloc[i]+=1
                if df['host'].iloc[j]=='novastore.farm.particle.cz':
                    df2['FZU'].iloc[i]+=1
        if df['tries'].iloc[j]=="3":
            if df['start1'].iloc[j]<=df2['index'].iloc[i] and df['end2'].iloc[j]>=df2['index'].iloc[i]:
                if df['cache'].iloc[j]=='root://hcc-stash.unl.edu':
                    df2['Nebraska'].iloc[i]+=1
                if df['cache'].iloc[j]=='root://mwt2-stashcache.campuscluster.illinois.edu':
                    df2['US-MWT2_UIUC'].iloc[i]+=1
                if df['cache'].iloc[j]=='root://xrd-cache-1.t2.ucsd.edu':
                    df2['UCSD'].iloc[i]+=1
                if df['cache'].iloc[j]=='root://osgxroot.usatlas.bnl.gov':
                    df2['BNL ATLAS'].iloc[i]+=1
                if df['cache'].iloc[j]=='novastore.farm.particle.cz':
                    df2['FZU'].iloc[i]+=1
                if df['cache'].iloc[j]=='root://data.ci-connect.net':
                    df2['Origin'].iloc[i]+=1
            if df['start3'].iloc[j]<=df2['index'].iloc[i] and df.index[j]>=df2['index'].iloc[i]:
                df2['Trunk'].iloc[i]+=1
            if df['end1'].iloc[j].replace(second=0, microsecond=0)==finish['index'].iloc[i].replace(second=0, microsecond=0):
                finish['BNL Timeout'].iloc[i]+=1
            if df['end2'].iloc[j].replace(second=0, microsecond=0)==finish['index'].iloc[i].replace(second=0, microsecond=0):
                finish['BNL Timeout'].iloc[i]+=1
        if df['tries'].iloc[j]=="2":
            if df['start2'].iloc[j]<=df2['index'].iloc[i] and df.index[j]>=df2['index'].iloc[i]:
                if df['cache'].iloc[j]=='root://hcc-stash.unl.edu':
                    df2['Nebraska'].iloc[i]+=1
                if df['cache'].iloc[j]=='root://mwt2-stashcache.campuscluster.illinois.edu':
                    df2['US-MWT2_UIUC'].iloc[i]+=1
                if df['cache'].iloc[j]=='root://xrd-cache-1.t2.ucsd.edu':
                    df2['UCSD'].iloc[i]+=1
                if df['cache'].iloc[j]=='root://osgxroot.usatlas.bnl.gov':
                    df2['BNL ATLAS'].iloc[i]+=1
                if df['cache'].iloc[j]=='novastore.farm.particle.cz':
                    df2['FZU'].iloc[i]+=1
                if df['cache'].iloc[j]=='root://data.ci-connect.net':
                    df2['Origin'].iloc[i]+=1
        if df.index[j].replace(second=0, microsecond=0)==finish['index'].iloc[i].replace(second=0, microsecond=0):
            if df['host'].iloc[j]=='root://hcc-stash.unl.edu':
                finish['Nebraska'].iloc[i]+=1
            if df['host'].iloc[j]=='root://mwt2-stashcache.campuscluster.illinois.edu':
                finish['US-MWT2_UIUC'].iloc[i]+=1
            if df['host'].iloc[j]=='root://xrd-cache-1.t2.ucsd.edu':
                finish['UCSD'].iloc[i]+=1
            if df['host'].iloc[j]=='root://osgxroot.usatlas.bnl.gov':
                finish['BNL ATLAS'].iloc[i]+=1
            if df['host'].iloc[j]=='novastore.farm.particle.cz':
                finish['FZU'].iloc[i]+=1
            if df['host'].iloc[j]=='root://data.ci-connect.net':
                if df['status'].iloc[j]=='Timeout':
                    finish['Trunk Timeout'].iloc[i]+=1
                else:
                    if df['tries'].iloc[j]=="1" or df['tries'].iloc[j]=="2":
                        finish['Origin'].iloc[i]+=1
                    elif df['tries'].iloc[j]=="3":
                        finish['Trunk'].iloc[i]+=1


                        


print('done')




/home/ivukotic/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  if __name__ == '__main__':


In [ ]:
#df.to_pickle('mwt2_total.pkl')
#df2.to_pickle('mwt2_concurrent.pkl')
#finish.to_pickle('undirected_finish.pkl')

#df2=pd.read_pickle('nebraska_concurrent.pkl')

## Plot concurrent jobs

In [ ]:
df2.plot.bar(stacked=True,figsize=[24, 12],color=['b', '#7FFFD4', 'g', 'y', 'k', 'm', 'c'])
plt.title('(5) Number of jobs running on each server per minute over a submit of 1,000 undirected jobs; timeout=1 mins', fontsize=20) 
plt.xlabel('Minutes', fontsize=15)
plt.ylabel('Count of concurrent jobs',fontsize=15)
#plt.text(90,95,'*No GLIDEIN_ResourceName specified in condor.submit')

plt.ylim([0,600])
plt.xticks(np.arange(0, length2+1, 5),np.arange(0, length2+1, 5))

#plt.savefig('undirected_1min_3.png')
plt.show()


## Plot finishing status of jobs

In [ ]:
finish.plot.bar(stacked=True,figsize=[24, 12],color=['b', '#7FFFD4', 'g', 'y', 'k', 'm', 'c', 'r', '#DDA0DD'])
plt.title('(5) Final status of jobs', fontsize=20) 
plt.xlabel('Minutes', fontsize=15)
plt.ylabel('Count of jobs',fontsize=15)
plt.xticks(np.arange(0, length2+1, 5),np.arange(0, length2+1, 5))

#plt.savefig('undirected_finish_1min_3.png')


plt.show()


## Plot counts of caches picked

In [ ]:
caches=df.groupby(['cache'])['cache'].count()
caches.plot.bar()
plt.title('(1) Count of first pick caches', fontsize=15) 
plt.xlabel('Caches', fontsize=15)
plt.ylabel('Count of jobs',fontsize=15)
plt.show()